# cli

> CLI tool for declarative plugin management

In [ ]:
#| default_exp cli

In [ ]:
#| export
import json
import os
import subprocess
import sys
from pathlib import Path
from typing import Optional

import typer
import yaml

app = typer.Typer(help="CJM Plugin System CLI", no_args_is_help=True)

@app.callback()
def main() -> None:
    """CJM Plugin System CLI for managing isolated plugin environments."""
    pass

In [ ]:
#| export
def run_cmd(
    cmd: str,  # Shell command to execute
    shell: bool = True,  # Whether to run through shell
    check: bool = True  # Whether to raise on non-zero exit
) -> None:
    """Run a shell command and stream output."""
    print(f"Running: {cmd}")
    subprocess.run(cmd, shell=shell, check=check, executable='/bin/bash')

In [ ]:
#| export
def _generate_manifest(
    env_name: str,  # Name of the Conda environment
    package_name: str,  # Package source string (git URL or package name)
    manifest_dir: Path  # Directory to write manifest JSON files
) -> None:
    """Run introspection script inside the target env to generate manifest."""
    print(f"[{env_name}] Generating manifest...")
    
    # Robust Module Name Extraction
    if package_name.startswith("git+"):
        # Case: git+https://github.com/cj-mills/repo-name.git
        url = package_name[4:]
        repo_part = url.split('/')[-1]
        module_name = repo_part.split('.git')[0].replace('-', '_')
    elif package_name.startswith("-e "):
        path = package_name.split("-e ")[1].strip()
        module_name = Path(path).name.replace('-', '_')
    elif "/" in package_name or "\\" in package_name:
        module_name = Path(package_name).name.replace('-', '_')
    else:
        module_name = package_name.replace('-', '_')

    print(f"[{env_name}] Introspecting module: {module_name}")
    
    # The introspection command
    # We explicitly redirect stderr to devnull to keep stdout clean for JSON
    introspection_cmd = (
        f"conda run -n {env_name} python -c "
        f"'from {module_name}.meta import get_plugin_metadata; "
        f"import json; print(json.dumps(get_plugin_metadata(), indent=2))'"
    )
    
    try:
        # Check output, capture stdout
        result_bytes = subprocess.check_output(introspection_cmd, shell=True, executable='/bin/bash')
        result_str = result_bytes.decode('utf-8').strip()
        
        # Robust JSON Parsing: 
        # Sometimes 'conda run' leaks warnings into stdout. We try to find the JSON block.
        try:
            start = result_str.find('{')
            end = result_str.rfind('}') + 1
            if start != -1 and end != 0:
                json_str = result_str[start:end]
                meta_json = json.loads(json_str)
            else:
                # Fallback to full string if brackets not found
                meta_json = json.loads(result_str)
        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to parse JSON from introspection output.")
            print(f"Raw Output:\n{result_str}")
            return

        plugin_name = meta_json.get('name', 'unknown')
        out_file = manifest_dir / f"{plugin_name}.json"
        
        with open(out_file, 'w') as f:
            f.write(json.dumps(meta_json, indent=2))
            
        print(f"[{env_name}] Wrote manifest to {out_file}")
        
    except subprocess.CalledProcessError as e:
        print(f"Error generating manifest for {env_name}:")
        print(e.output.decode() if e.output else str(e))
    except Exception as e:
        print(f"Unexpected error generating manifest: {e}")

In [ ]:
#| export
@app.command()
def install_all(
    config_path: str = typer.Option("plugins.yaml", "--config", help="Path to master config file"),
    force: bool = typer.Option(False, help="Force recreation of environments")
) -> None:
    """Install and register all plugins defined in plugins.yaml."""
    if not os.path.exists(config_path):
        typer.echo(f"Config file not found: {config_path}", err=True)
        raise typer.Exit(code=1)

    with open(config_path) as f:
        config = yaml.safe_load(f)

    # Setup manifest directory
    manifest_dir = Path.home() / ".cjm" / "plugins"
    manifest_dir.mkdir(parents=True, exist_ok=True)

    plugins = config.get('plugins', [])
    print(f"Found {len(plugins)} plugins to process.")

    for plugin in plugins:
        name = plugin.get('name')
        env_name = plugin.get('env_name')
        print(f"\n=== Processing {name} ({env_name}) ===")

        # 1. Check if Env Exists
        env_exists = False
        try:
            subprocess.check_call(
                f"conda env list | grep {env_name}", 
                shell=True, stdout=subprocess.DEVNULL
            )
            env_exists = True
        except subprocess.CalledProcessError:
            pass

        # 2. Create Environment
        if not env_exists or force:
            # SAFETY: If forcing, remove old one first to avoid flag issues
            if env_exists and force:
                print(f"Removing existing environment {env_name}...")
                run_cmd(f"conda env remove -n {env_name} -y")

            if 'env_file' in plugin:
                # NOTE: removed --force, added -n explicit override just in case
                # If the yaml file has a 'name' field, '-n' overrides it.
                run_cmd(f"conda env create -f {plugin['env_file']} -n {env_name}")
            elif 'python_version' in plugin:
                run_cmd(f"conda create -n {env_name} python={plugin['python_version']} -y")

        # 3. Install Dependencies (Pip)
        base_pip_cmd = f"conda run -n {env_name} pip install"
        
        if 'interface_libs' in plugin:
            libs = " ".join(plugin['interface_libs'])
            run_cmd(f"{base_pip_cmd} {libs}")

        if 'package' in plugin:
            run_cmd(f"{base_pip_cmd} {plugin['package']}")

        # 4. Generate Manifest
        pkg_source = plugin['package']
        _generate_manifest(env_name, pkg_source, manifest_dir)

    print("\n All operations complete.")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()